#⚙️ Stage 0: Configuration

In [23]:
import cv2
import numpy as np
import json
import torch
from transformers import AutoImageProcessor, AutoModel, pipeline
from PIL import Image
from scipy.ndimage import maximum_filter
import os # Import the 'os' module to check if files exist

# --- 1. Define Input File Paths ---
GOLDEN_IMAGE_PATH = "/content/Golden_Image.jpg"
CURRENT_IMAGE_PATH = "/content/Current_Image.jpg"

# --- Set benchmark path (or leave as "" if you don't have one) ---
BENCHMARK_IMAGE_PATH =  "" #"/content/BenchMark_Crack.jpg"

# --- 2. Define Output File Paths ---
OUTPUT_OVERLAY_PATH = "/content/sam2_output_overlay.png"
OUTPUT_REPORT_PATH = "/content/report.json"
OUTPUT_VALIDATION_PATH = "/content/validation_diff.jpg"

print("✅ Configuration loaded.")
print(f"   Golden Image: {GOLDEN_IMAGE_PATH}")
print(f"   Current Image: {CURRENT_IMAGE_PATH}")
print(f"   Benchmark: {BENCHMARK_IMAGE_PATH}")

✅ Configuration loaded.
   Golden Image: /content/Golden_Image.jpg
   Current Image: /content/Current_Image.jpg
   Benchmark: 


In [24]:
# import cv2
# import numpy as np
# import json
# import torch
# from transformers import AutoImageProcessor, AutoModel, pipeline
# from PIL import Image
# from scipy.ndimage import maximum_filter
# import os # Import the 'os' module to check if files exist

# # --- 1. Define Input File Paths ---
# # --- ⬇️ CORRECTION: Use the correct "before" and "after" pair ⬇️ ---
# GOLDEN_IMAGE_PATH = "/content/PCB_3_REF.jpg"
# CURRENT_IMAGE_PATH = "/content/PCB_4_CURR.jpg"


# # --- Set benchmark path (or leave as "" if you don't have one) ---
# BENCHMARK_IMAGE_PATH =  "" # No benchmark for this, so we leave it empty

# # --- 2. Define Output File Paths ---
# OUTPUT_OVERLAY_PATH = "/content/sam2_output_overlay.png"
# OUTPUT_REPORT_PATH = "/content/report.json"
# OUTPUT_VALIDATION_PATH = "/content/validation_diff.jpg"

# print("✅ Configuration loaded.")
# print(f"   Golden Image: {GOLDEN_IMAGE_PATH}")
# print(f"   Current Image: {CURRENT_IMAGE_PATH}")
# print(f"   Benchmark: {BENCHMARK_IMAGE_PATH}")

#1️⃣ Stage 1: ALIGN

In [25]:
# Load reference and current images
ref = cv2.imread(GOLDEN_IMAGE_PATH, cv2.IMREAD_COLOR)
curr = cv2.imread(CURRENT_IMAGE_PATH, cv2.IMREAD_COLOR)

# Detect keypoints and descriptors
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(ref, None)
kp2, des2 = sift.detectAndCompute(curr, None)

# Match keypoints
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

# Compute homography and warp
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
aligned = cv2.warpPerspective(curr, H, (ref.shape[1], ref.shape[0]))

print("✅ Stage 1 complete. Image aligned.")

✅ Stage 1 complete. Image aligned.


#2️⃣ Stage 2: DETECT (DINOv2)

In [26]:
# --- Load Model ---
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").to("cuda")

# --- Process Images ---
img_ref = Image.open(GOLDEN_IMAGE_PATH)
img_aligned = Image.fromarray(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))

inputs_ref = processor(img_ref, return_tensors="pt").to("cuda")
inputs_curr = processor(img_aligned, return_tensors="pt").to("cuda")

# --- Compute Features ---
with torch.no_grad():
    feats_ref = model(**inputs_ref).last_hidden_state
    feats_curr = model(**inputs_curr).last_hidden_state

# --- Exclude the [CLS] token ---
feats_ref_patches = feats_ref[:, 1:, :]
feats_curr_patches = feats_curr[:, 1:, :]

# --- Compute Cosine Similarity ---
cos_sim = torch.nn.functional.cosine_similarity(feats_ref_patches, feats_curr_patches, dim=-1)

# --- Reshape to 16x16 ---
heatmap_small = (1 - cos_sim[0]).reshape(16, 16).cpu().numpy()

# --- Resize heatmap ---
heatmap = cv2.resize(heatmap_small,
                     (aligned.shape[1], aligned.shape[0]),
                     interpolation=cv2.INTER_LINEAR)

print("✅ Stage 2 complete. Full-size heatmap generated.")

✅ Stage 2 complete. Full-size heatmap generated.


#3️⃣ Stage 3: SEGMENT (SAM 2)

In [27]:
# # --- Initialize Model ---
# mask_generator = pipeline(
#     "mask-generation",
#     model="facebook/sam2-hiera-large",
#     device_map="cuda"
# )

# # --- Convert Image ---
# image_pil = Image.fromarray(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))

# # --- IMPROVED PROMPT GENERATION (WITH BLURRING) ---

# # 1. Normalize heatmap
# heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# # 2. De-noise the heatmap to remove speckles
# heatmap_blurred = cv2.GaussianBlur(heatmap_norm, (11, 11), 0)

# # 3. Find ALL local peaks on the *blurred* heatmap
# footprint = np.ones((20, 20)) # Neighborhood size
# local_max = maximum_filter(heatmap_blurred, footprint=footprint)
# peaks_mask = (heatmap_blurred == local_max)

# # 4. Apply a threshold to the peaks (adjust '40' if needed)
# peaks_mask = np.logical_and(peaks_mask, heatmap_blurred > 40)

# # 5. Get the (y, x) coordinates of these peaks
# y_coords, x_coords = np.where(peaks_mask)

# # 6. Format them as [x, y] points for SAM
# points = [[x, y] for x, y in zip(x_coords, y_coords)]
# labels = [1] * len(points) # '1' means foreground

# if not points:
#     print("⚠️ No peaks found above the threshold. Try lowering it.")
# else:
#     print(f"✅ Found {len(points)} precise peak prompts from heatmap.")

# # --- Generate Masks using POINT prompts ---
# if points:
#     results = mask_generator(
#         image_pil,
#         points=[points], # Note the extra list wrapping
#         labels=[labels],
#         boxes=None
#     )
#     masks = results["masks"]
# else:
#     masks = []

# print(f"✅ SAM 2 generated {len(masks)} masks from point prompts.")

# # --- Visualization Loop ---
# overlay = aligned.copy()
# color = [0, 0, 255] # Red (BGR)
# target_dsize = (aligned.shape[1], aligned.shape[0])

# for m in masks:
#     m_np = np.array(m).astype(np.uint8) * 255
#     m_resized = cv2.resize(m_np, target_dsize, interpolation=cv2.INTER_NEAREST)
#     m_bin = m_resized > 0
#     overlay[m_bin] = color

# # --- Save as lossless PNG ---
# cv2.imwrite(OUTPUT_OVERLAY_PATH, overlay)
# print(f"✅ Segmentation complete — saved overlay to {OUTPUT_OVERLAY_PATH}")

Improved Stage-3

In [28]:
# from transformers import pipeline
# import torch
# from PIL import Image
# import numpy as np
# import cv2
# from scipy.ndimage import maximum_filter

# # --- Initialize Model ---
# mask_generator = pipeline(
#     "mask-generation",
#     model="facebook/sam2-hiera-large",
#     device_map="cuda"
# )

# # --- Convert Image ---
# image_pil = Image.fromarray(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))

# # --- IMPROVED PROMPT GENERATION (WITH BLURRING) ---

# # 1. Normalize heatmap
# heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# # 2. De-noise the heatmap
# heatmap_blurred = cv2.GaussianBlur(heatmap_norm, (11, 11), 0)

# # 3. Find ALL local peaks on the *blurred* heatmap
# footprint = np.ones((20, 20)) # Neighborhood size
# local_max = maximum_filter(heatmap_blurred, footprint=footprint)
# peaks_mask = (heatmap_blurred == local_max)

# # 4. Apply a threshold to the peaks
# # --- ⬇️ CORRECTION: Increased threshold to filter out noise ⬇️ ---
# peaks_mask = np.logical_and(peaks_mask, heatmap_blurred > 80)

# # 5. Get the (y, x) coordinates of these peaks
# y_coords, x_coords = np.where(peaks_mask)

# # 6. Format them as [x, y] points for SAM
# points = [[x, y] for x, y in zip(x_coords, y_coords)]
# labels = [1] * len(points) # '1' means foreground

# if not points:
#     print("✅ Found 0 peaks. This is the correct result for identical images.")
# else:
#     print(f"✅ Found {len(points)} precise peak prompts from heatmap.")

# # --- Generate Masks using POINT prompts ---
# if points:
#     results = mask_generator(
#         image_pil,
#         points=[points], # Note the extra list wrapping
#         labels=[labels],
#         boxes=None
#     )
#     masks = results["masks"]
# else:
#     masks = []

# print(f"✅ SAM 2 generated {len(masks)} masks from point prompts.")

# # --- Visualization Loop ---
# overlay = aligned.copy()
# color = [0, 0, 255] # Red (BGR)
# target_dsize = (aligned.shape[1], aligned.shape[0])

# for m in masks:
#     m_np = np.array(m).astype(np.uint8) * 255
#     m_resized = cv2.resize(m_np, target_dsize, interpolation=cv2.INTER_NEAREST)
#     m_bin = m_resized > 0
#     overlay[m_bin] = color

# # --- Save as lossless PNG ---
# cv2.imwrite(OUTPUT_OVERLAY_PATH, overlay)
# print(f"✅ Segmentation complete — saved overlay to {OUTPUT_OVERLAY_PATH}")

Higher Threshold Stage-3


In [29]:
# from transformers import pipeline
# import torch
# from PIL import Image
# import numpy as np
# import cv2
# from scipy.ndimage import maximum_filter

# # --- Initialize Model ---
# mask_generator = pipeline(
#     "mask-generation",
#     model="facebook/sam2-hiera-large",
#     device_map="cuda"
# )

# # --- Convert Image ---
# image_pil = Image.fromarray(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))

# # --- IMPROVED PROMPT GENERATION (WITH BLURRING) ---

# # 1. Normalize heatmap
# heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# # 2. De-noise the heatmap
# heatmap_blurred = cv2.GaussianBlur(heatmap_norm, (11, 11), 0)

# # 3. Find ALL local peaks on the *blurred* heatmap
# footprint = np.ones((20, 20)) # Neighborhood size
# local_max = maximum_filter(heatmap_blurred, footprint=footprint)
# peaks_mask = (heatmap_blurred == local_max)

# # 4. Apply a threshold to the peaks
# # --- ⬇️ CORRECTION: Drastically Increased threshold ⬇️ ---
# peaks_mask = np.logical_and(peaks_mask, heatmap_blurred > 150) # Was 80

# # 5. Get the (y, x) coordinates of these peaks
# y_coords, x_coords = np.where(peaks_mask)

# # 6. Format them as [x, y] points for SAM
# points = [[x, y] for x, y in zip(x_coords, y_coords)]
# labels = [1] * len(points) # '1' means foreground

# if not points:
#     print("✅ Found 0 peaks. This is the correct result for identical images.")
# else:
#     print(f"✅ Found {len(points)} precise peak prompts from heatmap.")

# # --- Generate Masks using POINT prompts ---
# if points:
#     results = mask_generator(
#         image_pil,
#         points=[points], # Note the extra list wrapping
#         labels=[labels],
#         boxes=None
#     )
#     masks = results["masks"]
# else:
#     masks = []

# print(f"✅ SAM 2 generated {len(masks)} masks from point prompts.")

# # --- Visualization Loop ---
# overlay = aligned.copy()
# color = [0, 0, 255] # Red (BGR)
# target_dsize = (aligned.shape[1], aligned.shape[0])

# for m in masks:
#     m_np = np.array(m).astype(np.uint8) * 255
#     m_resized = cv2.resize(m_np, target_dsize, interpolation=cv2.INTER_NEAREST)
#     m_bin = m_resized > 0
#     overlay[m_bin] = color

# # --- Save as lossless PNG ---
# cv2.imwrite(OUTPUT_OVERLAY_PATH, overlay)
# print(f"✅ Segmentation complete — saved overlay to {OUTPUT_OVERLAY_PATH}")

In [30]:
# # --- DEBUG: Save heatmaps for visualization ---
# cv2.imwrite("/content/DEBUG_heatmap_normalized.png", heatmap_norm)
# cv2.imwrite("/content/DEBUG_heatmap_blurred.png", heatmap_blurred)

# print("✅ Debug heatmaps saved. Please check:")
# print("   - /content/DEBUG_heatmap_normalized.png")
# print("   - /content/DEBUG_heatmap_blurred.png")

In [31]:
from transformers import pipeline
import torch
from PIL import Image
import numpy as np
import cv2
from scipy.ndimage import maximum_filter

# --- Initialize Model ---
mask_generator = pipeline(
    "mask-generation",
    model="facebook/sam2-hiera-large",
    device_map="cuda"
)

# --- Convert Image ---
image_pil = Image.fromarray(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))

# --- IMPROVED PROMPT GENERATION (WITH STRONGER FILTERING) ---

# 1. Normalize heatmap
heatmap_norm = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# 2. De-noise the heatmap more aggressively
# --- ⬇️ Increased Gaussian Blur Kernel Size ⬇️ ---
heatmap_blurred = cv2.GaussianBlur(heatmap_norm, (21, 21), 0) # Was (11, 11)

# 3. Find ALL local peaks on the *blurred* heatmap
footprint = np.ones((20, 20)) # Neighborhood size (keep as is for now)
local_max = maximum_filter(heatmap_blurred, footprint=footprint)
peaks_mask = (heatmap_blurred == local_max)

# 4. Apply a much higher threshold to the peaks
# --- ⬇️ Drastically Increased threshold ⬇️ ---
peaks_mask = np.logical_and(peaks_mask, heatmap_blurred > 200) # Was 150

# 5. Get the (y, x) coordinates of these peaks
y_coords, x_coords = np.where(peaks_mask)

# 6. Format them as [x, y] points for SAM
points = [[x, y] for x, y in zip(x_coords, y_coords)]
labels = [1] * len(points) # '1' means foreground

if not points:
    # This is the expected output now
    print("✅ Found 0 peaks. This is the correct result for identical images.")
else:
    # If peaks are still found, the noise is incredibly high
    print(f"⚠️ Found {len(points)} peaks even with strong filtering.")
    print(f"   Consider alternative alignment or comparison methods if noise persists.")


# --- Generate Masks using POINT prompts ---
if points:
    results = mask_generator(
        image_pil,
        points=[points], # Note the extra list wrapping
        labels=[labels],
        boxes=None
    )
    masks = results["masks"]
else:
    masks = [] # No points means no masks

print(f"✅ SAM 2 generated {len(masks)} masks from point prompts.")

# --- Visualization Loop ---
overlay = aligned.copy()
color = [0, 0, 255] # Red (BGR)
target_dsize = (aligned.shape[1], aligned.shape[0])

# This loop won't run if masks is empty
for m in masks:
    m_np = np.array(m).astype(np.uint8) * 255
    m_resized = cv2.resize(m_np, target_dsize, interpolation=cv2.INTER_NEAREST)
    m_bin = m_resized > 0
    overlay[m_bin] = color

# --- Save as lossless PNG ---
cv2.imwrite(OUTPUT_OVERLAY_PATH, overlay)
print(f"✅ Segmentation complete — saved overlay to {OUTPUT_OVERLAY_PATH}")

# --- Also save the heatmaps for debugging ---
cv2.imwrite("/content/DEBUG_heatmap_normalized.png", heatmap_norm)
cv2.imwrite("/content/DEBUG_heatmap_blurred_strong.png", heatmap_blurred) # New name
print("✅ Debug heatmaps saved.")

Device set to use cuda


⚠️ Found 17 peaks even with strong filtering.
   Consider alternative alignment or comparison methods if noise persists.
✅ SAM 2 generated 122 masks from point prompts.
✅ Segmentation complete — saved overlay to /content/sam2_output_overlay.png
✅ Debug heatmaps saved.


#4️⃣ Stage 4: REPORT (JSON)

In [32]:
# --- Cast np.int64 to standard python int() ---
report_points = [{"x": int(p[0]), "y": int(p[1])} for p in points]

report = {
    "num_differences_found": len(report_points),
    "prompt_type": "points",
    "differences": report_points
}

with open(OUTPUT_REPORT_PATH, "w") as f:
    json.dump(report, f, indent=2)

print(f"✅ JSON report saved to {OUTPUT_REPORT_PATH}")

✅ JSON report saved to /content/report.json


#5️⃣ Stage 5: EVALUATE (IoU)

In [33]:
print("🚀 Starting Stage 5: Benchmark Validation")

# --- 1. Define File Paths from Stage 0 ---
ref = cv2.imread(GOLDEN_IMAGE_PATH, cv2.IMREAD_COLOR)
predicted_overlay_path = OUTPUT_OVERLAY_PATH
benchmark_path = BENCHMARK_IMAGE_PATH
validation_output_path = OUTPUT_VALIDATION_PATH
report_path = OUTPUT_REPORT_PATH

# --- 2. ⬇️ MODIFIED: Check if Benchmark File Exists ⬇️ ---
if not os.path.exists(benchmark_path):
    print(f"⚠️ Benchmark file not found at: {benchmark_path}")
    print(f"   Skipping evaluation. Your output is saved at: {OUTPUT_OVERLAY_PATH}")

else:
    # --- 3. Load Images (Benchmark exists, so proceed) ---
    print(f"✅ Benchmark found at {benchmark_path}. Running evaluation...")
    try:
        predicted_overlay = cv2.imread(predicted_overlay_path)
        if predicted_overlay is None:
            raise IOError(f"Could not load predicted overlay from {predicted_overlay_path}")

        benchmark_img = cv2.imread(benchmark_path, cv2.IMREAD_GRAYSCALE)
        if benchmark_img is None:
            raise IOError(f"Could not load benchmark image from {benchmark_path}")

        print("✅ Images loaded successfully.")

    except Exception as e:
        print(f"❌ ERROR: {e}")
        print(f"Please ensure '{predicted_overlay_path}' and '{benchmark_path}' are valid image files.")

    else:
        # --- 4. Resize Benchmark to Match Output ---
        target_shape = (ref.shape[1], ref.shape[0]) # (width, height)
        if (benchmark_img.shape[1], benchmark_img.shape[0]) != target_shape:
            print(f"Resizing benchmark from {benchmark_img.shape} to {target_shape}")
            benchmark_img = cv2.resize(benchmark_img, target_shape, interpolation=cv2.INTER_NEAREST)

        # --- 5. Create Boolean Masks ---
        predicted_mask = (predicted_overlay[:, :, 2] > 200) & \
                         (predicted_overlay[:, :, 1] < 50) & \
                         (predicted_overlay[:, :, 0] < 50)

        # --- CRITICAL BENCHMARK FIX ---
        # Assumes benchmark is BLACK crack on WHITE background
        ground_truth_mask = benchmark_img < 100

        # --- 6. Calculate Metrics (IoU) ---
        intersection = np.logical_and(predicted_mask, ground_truth_mask)
        union = np.logical_or(predicted_mask, ground_truth_mask)

        if np.sum(union) > 0:
            iou_score = np.sum(intersection) / np.sum(union)
        else:
            iou_score = 1.0 if np.sum(intersection) == 0 else 0.0

        print(f"---" * 10)
        print(f"📊 Validation Score (IoU): {iou_score:.4f}")
        print(f"---" * 10)

        # --- 7. Create Visual Validation "Diff" Image ---
        validation_image = ref.copy()
        TP_COLOR = [255, 0, 255]  # Magenta (True Positive)
        FP_COLOR = [0, 0, 255]    # Red (False Positive)
        FN_COLOR = [255, 255, 0]  # Cyan (False Negative)

        tp_mask = intersection
        fp_mask = np.logical_and(predicted_mask, np.logical_not(ground_truth_mask))
        fn_mask = np.logical_and(np.logical_not(predicted_mask), ground_truth_mask)

        validation_image[tp_mask] = TP_COLOR
        validation_image[fp_mask] = FP_COLOR
        validation_image[fn_mask] = FN_COLOR

        cv2.imwrite(validation_output_path, validation_image)
        print(f"✅ Validation 'diff' image saved to {validation_output_path}")

        # --- 8. (Optional) Update JSON Report with IoU Score ---
        try:
            with open(report_path, "r") as f:
                report_data = json.load(f)

            report_data["validation_score_IoU"] = iou_score

            with open(report_path, "w") as f:
                json.dump(report_data, f, indent=2)

            print(f"✅ JSON report updated with IoU score.")

        except Exception as e:
            print(f"⚠️ Warning: Could not update JSON report. {e}")

🚀 Starting Stage 5: Benchmark Validation
⚠️ Benchmark file not found at: 
   Skipping evaluation. Your output is saved at: /content/sam2_output_overlay.png
